In [63]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import random
import time

from fairdivision.algorithms.ordered_picking import ordered_picking
from fairdivision.experiments.measurements import *
from fairdivision.utils.allocation import Allocation
from fairdivision.utils.checkers import *
from fairdivision.utils.generators import *

In [66]:
def run_algorithm(n, m, generator, iterations, algorithm, property_name):
    seed = random.randint(1, 100_000_000)
    random.seed(seed)
    
    measurements = []
    
    for _ in range(iterations):
        agents = generate_agents(n)
        items = generate_items(m)
        
        generate_valuations(agents, items, generator)
        
        allocation = algorithm(agents, items)

        if property_name == "EF1":
            measurements.append((ef1_satisfied_factor(agents, allocation), highest_ef1_approximation(agents, allocation)))
        elif property_name == "EFX":
            measurements.append((efx_satisfied_factor(agents, allocation), highest_efx_approximation(agents, allocation)))

    return measurements


def print_property_satisfaction(n, m, measurements, property_name):
    not_satisfied = list(filter(lambda measurement: measurement[1] < 1.0, measurements))

    factors = dict()
    approximations = list()

    for factor, _ in not_satisfied:
        if factor in factors:
            factors[factor] += 1
        else:
            factors[factor] = 1

    print(f"{n} agents and {m} items, {len(not_satisfied)} out of {len(measurements)} not {property_name}\n  factors:        [", end="")
    
    first = True
    for factor, occurrences in sorted(factors.items(), key=lambda pair: pair[0]):
        if not first:
            print(", ", end="")

        if occurrences == 1:
            print(f"{factor} - {occurrences} time", end="")
        else:
            print(f"{factor} - {occurrences} times", end="")

        first = False

    print("]\n  approximations: [", end="")

    first = True
    for _, approximation in sorted(filter(lambda measurement: measurement[1] < 1.0, measurements), reverse=True):
        if not first:
            print(", ", end="")

        print(f"{approximation}", end="")
        
        first = False

    print("]\n")

In [67]:
n = 5
ms = np.concatenate((np.arange(2, 100, 2), np.arange(100, 1050, 50)))

generator = AdditiveGenerator(min=0, max=100)
# generator = OrderedGenerator(min=0, max=10000)

iterations = 100

for m in ms:
    measurements = run_algorithm(n, m, generator, iterations, ordered_picking, "EFX")
    print_property_satisfaction(n, m, measurements, "EFX")

5 agents and 2 items, 0 out of 100 not EFX
  factors:        []
  approximations: []

5 agents and 4 items, 0 out of 100 not EFX
  factors:        []
  approximations: []

5 agents and 6 items, 1 out of 100 not EFX
  factors:        [0.8 - 1 time]
  approximations: [0.733]

5 agents and 8 items, 26 out of 100 not EFX
  factors:        [0.2 - 1 time, 0.4 - 6 times, 0.6 - 8 times, 0.8 - 11 times]
  approximations: [0.986, 0.948, 0.946, 0.753, 0.698, 0.686, 0.658, 0.645, 0.603, 0.595, 0.586, 0.867, 0.817, 0.718, 0.693, 0.68, 0.672, 0.662, 0.559, 0.886, 0.848, 0.687, 0.655, 0.537, 0.517, 0.729]

5 agents and 10 items, 37 out of 100 not EFX
  factors:        [0.6 - 12 times, 0.8 - 25 times]
  approximations: [0.988, 0.979, 0.971, 0.967, 0.965, 0.938, 0.923, 0.903, 0.845, 0.843, 0.829, 0.807, 0.795, 0.772, 0.772, 0.758, 0.752, 0.683, 0.665, 0.664, 0.653, 0.649, 0.618, 0.584, 0.574, 0.946, 0.881, 0.833, 0.782, 0.764, 0.746, 0.692, 0.687, 0.683, 0.619, 0.562, 0.555]

5 agents and 12 items, 23 

KeyboardInterrupt: 